In [1]:
import sys, os, pickle, time, json, yaml, re
from glob import glob
from prettytable import PrettyTable

In [2]:
results_dir = os.environ['HOME'] + '/public_html/BPH_RDst/Combine/'
version = '13'
cat = 'comb'

In [3]:
dirs = glob(results_dir + '*'+version+'*'+cat+'*')
for d in dirs:
    if (not os.path.basename(d).startswith('v'+version+'_')) and (not os.path.basename(d).startswith(version+'_')):
        print os.path.basename(d)
        dirs.remove(d)

summary_table_13_comb.txt


In [4]:
table = PrettyTable()
table.field_names = ['Version', 'Sat pval', 'Scan [%]', 'Cat comp', 'Top pulls','[sigma]']
table.align['Top pulls'] = 'l'
table.align['[sigma]'] = 'r'

for dd in sorted(dirs):
    print os.path.basename(dd)
    
    tag = os.path.basename(dd)
    tag = tag[tag.find(version+'_')+len(version+'_'):]
    tag = tag[:tag.find(cat)]
    if tag.endswith('_'):
        tag = tag[:-1]
#     print tag


    GoF_file = dd + '/GoF_results.txt'
    pval_sat = '-'
    if os.path.isfile(GoF_file):
        with open(GoF_file) as f:
            for line in reversed(f.readlines()):
                if 'algoSat' in line:
                    break
            data = [x for x in line.split(' ') if x]
            data[2] = data[2][1:-2]
            if float(data[2]) > 0:
                pval_sat = data[2] + '%'
            else:
                pval_sat = 'x{:.2f}'.format(float(data[1])/float(data[-2]))
#     print pval_sat


    scan_file = dd + '/scan_results.txt'
    scan_result = '-'
    if os.path.isfile(scan_file):
        with open(scan_file) as f:
            line = f.readlines()[-1]
            data = [x for x in line.split(' ') if x]
            err = float(data[2][1:]) + float(data[4][1:])
            scan_result = '{:.1f} +/- {:.1f}'.format(100*float(data[1]), 100*err*0.5)
#     print scan_result

    catComp_file = dd + '/categoriesCompatibility.txt'
    catComp = '-'
    if os.path.isfile(catComp_file):
        with open(catComp_file) as f:
            line = f.readlines()[-1]
            catComp = line.split(' ')[-1][1:-5] + '%'
#     print catComp

    pulls_file = dd + '/scanNuisanceOut_Base.txt'
    nPulls = 5
    pulls = ['-']*nPulls
    if os.path.isfile(pulls_file):
        with open(pulls_file) as f:
            for i, line in enumerate(f.readlines()[3:nPulls+3]):
                pulls[i] = [x for x in line[:-1].replace('|', '').split(' ') if x]
#     for p in pulls:
#         print p

    if dd != dirs[0]:
        table.add_row(len(table.field_names)*[''])
    table.add_row([tag, pval_sat, scan_result, catComp, pulls[0][0], pulls[0][1]])
    for i in range(1, len(pulls)):
        table.add_row(['', '', '', '', pulls[i][0], pulls[i][1]])

13_base_comb_CLN_blinded
v13_M2miss_0p0__mVisPM_3p8_comb_CLN_blinded
v13_M2miss_0p8__mVisPM_3p8_comb_CLN_blinded
v13_corr_tkPVfrac_comb_CLN_blinded
v13_mHadPM_2p6_comb_CLN_blinded
v13_mVisPM_3p6_comb_CLN_blinded
v13_mVisPM_3p8_mHadPM_2p6_comb_CLN_blinded
v13_noLostHits_comb_CLN_blinded
v13_noLostHits_mVisPM_3p8_comb_CLN_blinded
v13_tkPval20_comb_CLN_blinded


In [5]:
print table

+------------------------+----------+--------------+----------+---------------------------------+---------+
|        Version         | Sat pval |   Scan [%]   | Cat comp | Top pulls                       | [sigma] |
+------------------------+----------+--------------+----------+---------------------------------+---------+
|          base          |  x1.12   | 37.9 +/- 9.1 |   71%    | B2DstCLNeig2                    |    3.17 |
|                        |          |              |          | brBu_DstDustK                   |    2.93 |
|                        |          |              |          | brBd_DstDustK                   |    2.25 |
|                        |          |              |          | prop_binhigh_ctrl_pm_mVis_bin15 |   -2.11 |
|                        |          |              |          | brDst2S_DstPiPi                 |    2.07 |
|                        |          |              |          |                                 |         |
| M2miss_0p0__mVisPM_3p8 |  

In [6]:
outfile = results_dir + 'summary_table_{}_{}.txt'.format(version, cat)
os.system('date > '+outfile)
with open(outfile, 'a') as f:
    f.write(table.get_string())
    
# with open(outfile[:-4]+'.html', 'w') as f:
#     f.write(table.get_html_string())